In [3]:
import pickle
import gradio as gr
import pandas as pd
import numpy as np

C:\Users\Mehmet Arif\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
filepath = 'stroke_model.pkl'
model = pickle.load(open(filepath, 'rb'))

In [5]:


def predict(name,gender,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,weight,height,smoking_status):

    if not avg_glucose_level:
      avg_glucose_level = "99"

    if gender == "Male":
        gender = int(1)
    else:
        gender = int(0)
    
    hypertension_cv = int(hypertension == True)
    ever_married_cv = int(ever_married == True)
    heart_disease_cv = int(heart_disease == True)
    
    if work_type == "Private":
        work_type = 2
    elif work_type == "Self-Employed":
        work_type = 4
    elif work_type == "Children":
        work_type = 1
    elif work_type == "Government Job":
        work_type = 3
    elif work_type == "Never Worked":
        work_type = 0
        
    bmi = float(weight)/ float(height)**2
    
    if bmi > 30:
        obese = 1
    else:
        obese =0
    if bmi > 40:
        morbid = 1
    else:
        morbid = 0
      
    risk_factors = 0 + (1 if float(avg_glucose_level) > 99 else 0) + \
              (1 if age > 45 else 0) + \
              (1 if bmi > 24.99 else 0) + \
              (1 if hypertension == 1 else 0) + \
              (1 if heart_disease == 1 else 0) + \
              (1 if smoking_status in ['Formerly Smoked', 'Smokes'] else 0)
    smoking_status_Unknown = 0
    smoking_status_formerly = 0
    smoking_status_never = 0
    smoking_status_smokes = 0

    if smoking_status == "I Don't Wanna Say":
        smoking_status_Unknown = 1
    elif smoking_status == "Never Smoked":
        smoking_status_never = 1
    elif smoking_status == "Formerly Smoked":
        smoking_status_formerly = 1
    elif smoking_status == "Smokes":
        smoking_status_smokes = 1
    

    df_pred = pd.DataFrame({    
        "gender": [gender],
        "age" : [float(age)],
        "hypertension": [hypertension_cv],
        "ever_married": [ever_married_cv],
        "heart_disease" : [heart_disease_cv],
        "work_type": [work_type],
        "avg_glucose_level": [float(avg_glucose_level)],
        "bmi": [bmi],
        "morbid" : [morbid],
        "obese": [obese],
        "risk_factors" : [risk_factors],
        "smoking_status_Unknown" : [np.uint8(smoking_status_Unknown)],
        "smoking_status_formerly smoked": [np.uint8(smoking_status_formerly)],
        "smoking_status_never smoked" : [np.uint8(smoking_status_never)],
        "smoking_status_smokes" : [np.uint8(smoking_status_smokes)]
    })
    valu = model.predict_proba(df_pred)[:, 1][0]
        
    return "Hello Dear " + name +\
", Welcome\n\n" + "Your risk rate is: %" + str(valu*100) + "\n\n" +\
("Your risk of stroke is below the average.\n\n For our other services you can contact with us via 444 04 44 " if valu < 0.037492 else "") +\
("Your risk of stroke is above the average! Please contact with our hospital to check up \n\nYou can contact via 444 04 44 " if valu >= 0.037492 else "")+\
"\n\nHave a healthy day\n"

demo = gr.Interface(fn=predict,
                    inputs=[gr.Textbox(label = "Your name"),
                            gr.Radio(["Male","Female"], label = "Your gender", info = "Are you XX or XY"),
                            gr.Slider(0, 120, label = "How old are you?"),
                            gr.Checkbox(label = "I have hypertension"),
                            gr.Checkbox(label = "I have a heart disease"),
                            gr.Checkbox(label = "I have married at least once"),
                            gr.Radio(["Private","Self-Employed","Children","Government Job", "Never Worked"], label= "What's your work type"),
                            gr.Textbox(label = "Your average glucose level",placeholder = "It is set 99 mg/dL by default" ,info = " If you don't know your average glucose level, \
                            you can leave blank here. However, it is highly recommended to get information about it for a better result. You can contact with us via 444 04 44 to \
                            find out your exact average glucose level."),
                            gr.Textbox(label = "Your body weight", info = "In kilograms"),
                            gr.Textbox(label = "Your body height", info = "In meters e.g. 1.70" ),
                            gr.Radio(["I Don't Wanna Say", "Never Smoked", "Formerly Smoked", "Smokes"], label = "Your smoking status",
                                     info = "Smoking kills")
                            ],
                    outputs = gr.Textbox(label = "Result"),
                    #css="body {background-color: blue}",
                    title = "IKC™ Hospital Stroke Prediction Service",
                    article = "Provided by Insafsız Kudret Cemiyeti™",
                    theme=gr.themes.Default(primary_hue=gr.themes.colors.pink, secondary_hue=gr.themes.colors.rose)
                   )
                    
demo.launch(share = True)

#gender: male,female
#age: int
#hypertension: 0,1
#heart_disease: 0,1
#ever_married: Yes,No
#work_type: 0,1,2,3,4
#avg_glucose_level: int
#bmi: int
#smoking_status: "Unknown", "Never Smoked", "Formerly Smoked", "Smokes"


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app
